[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/gpt-4-langchain-docs.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/gpt-4-langchain-docs.ipynb)

# GPT4 with Retrieval Augmentation over LangChain Docs

In this notebook we'll work through an example of using GPT-4 with retrieval augmentation to answer questions about the LangChain Python library.

[![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/full-link.svg)](https://github.com/pinecone-io/examples/blob/master/learn/generation/openai/gpt-4-langchain-docs.ipynb)

To begin we must install the prerequisite libraries:

In [1]:
!pip install -qU \
  openai==0.27.7 \
  "pinecone-client[grpc]"==2.2.1 \
  pinecone-datasets=='0.5.0rc11'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.0/223.0 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

---

🚨 _Note: the above `pip install` is formatted for Jupyter notebooks. If running elsewhere you may need to drop the `!`._

---

In this example, we will use a pre-embedding dataset of the LangChain docs from [python.langchain.readthedocs.com/](https://python.langchain.com/en/latest/). If you'd like to see how we perform the data preparation refer to [this notebook]().

Let's go ahead and download the dataset.

In [21]:
from pinecone_datasets import load_dataset

dataset = load_dataset('langchain-python-docs-text-embedding-ada-002')
# we drop sparse_values as they are not needed for this example
dataset.documents.drop(['metadata'], axis=1, inplace=True)
dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)
dataset.head()

/usr/local/lib/python3.10/dist-packages/pinecone_datasets/dataset.py:125: UserWarning: WARNING: No data found at: gs://pinecone-datasets-dev/langchain-python-docs-text-embedding-ada-002/queries/*.parquet. Returning empty DF
  warnings.warn(


,id,values,metadata
0,417ede5d-39be-498f-b518-f47ed4e53b90,"[0.005949743557721376, 0.01983247883617878, -0...","{'chunk': 0, 'text': '.rst .pdf Welcome to Lan..."
1,110f550d-110b-4378-b95e-141397fa21bc,"[0.009401749819517136, 0.02443608082830906, 0....","{'chunk': 1, 'text': 'Use Cases# Best practice..."
2,d5f00f02-3295-4567-b297-5e3262dc2728,"[-0.005517194513231516, 0.0208403542637825, 0....","{'chunk': 2, 'text': 'Gallery: A collection of..."
3,0b6fe3c6-1f0e-4608-a950-43231e46b08a,"[-0.006499645300209522, 0.0011573900701478124,...","{'chunk': 0, 'text': 'Search Error Please acti..."
4,39d5f15f-b973-42c0-8c9b-a2df49b627dc,"[-0.005658374633640051, 0.00817849114537239, 0...","{'chunk': 0, 'text': '.md .pdf Dependents Depe..."


Our chunks are ready so now we move onto embedding and indexing everything.

## Initializing the Index

Now we need a place to store these embeddings and enable a efficient vector search through them all. To do that we use Pinecone, we can get a [free API key](https://app.pinecone.io/) and enter it below where we will initialize our connection to Pinecone and create a new index.

In [17]:
import os
import pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.getenv("PINECONE_API_KEY") or "PINECONE_API_KEY"
# find your environment next to the api key in pinecone console
env = os.getenv("PINECONE_ENVIRONMENT") or "PINECONE_ENVIRONMENT"

pinecone.init(api_key=api_key, environment=env)

In [18]:
index_name = 'gpt-4-langchain-docs-fast'

In [19]:
import time

# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=1536,  # dimensionality of text-embedding-ada-002
        metric='cosine'
    )
    # wait for index to be initialized
    time.sleep(1)

# connect to index
index = pinecone.GRPCIndex(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

We can see the index is currently empty with a `total_vector_count` of `0`. We can begin populating it with OpenAI `text-embedding-ada-002` built embeddings like so:

In [23]:
for batch in dataset.iter_documents(batch_size=100):
    index.upsert(batch)

sending upsert requests:   0%|          | 0/3476 [00:00<?, ?it/s]

collecting async responses:   0%|          | 0/35 [00:00<?, ?it/s]

upserted_count: 3476

Now we've added all of our langchain docs to the index. With that we can move on to retrieval and then answer generation using GPT-4.

## Retrieval

To search through our documents we first need to create a query vector `xq`. Using `xq` we will retrieve the most relevant chunks from the LangChain docs. To create that query vector we must initialize a `text-embedding-ada-002` embedding model with OpenAI. For this, you need an [OpenAI API key](https://platform.openai.com/).

In [25]:
import openai

# get api key from platform.openai.com
openai.api_key = os.getenv('OPENAI_API_KEY') or 'OPENAI_API_KEY'

embed_model = "text-embedding-ada-002"

In [26]:
query = "how do I use the LLMChain in LangChain?"

res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(xq, top_k=5, include_metadata=True)

In [27]:
res

{'matches': [{'id': '2f66a6a5-c829-4118-acb8-f08667f3f95d',
              'metadata': {'chunk': 2.0,
                           'text': 'for full documentation on:\\n\\nGetting '
                                   'started (installation, setting up the '
                                   'environment, simple examples)\\n\\nHow-To '
                                   'examples (demos, integrations, helper '
                                   'functions)\\n\\nReference (full API '
                                   'docs)\\n\\nResources (high-level '
                                   'explanation of core '
                                   'concepts)\\n\\nð\\x9f\\x9a\\x80 What can '
                                   'this help with?\\n\\nThere are six main '
                                   'areas that LangChain is designed to help '
                                   'with.\\nThese are, in increasing order of '
                                   'complexity:\\n\\nð\\x9f“\\x83 LLMs

With retrieval complete, we move on to feeding these into GPT-4 to produce answers.

## Retrieval Augmented Generation

GPT-4 is currently accessed via the `ChatCompletions` endpoint of OpenAI. To add the information we retrieved into the model, we need to pass it into our user prompts *alongside* our original query. We can do that like so:

In [28]:
# get list of retrieved text
contexts = [item['metadata']['text'] for item in res['matches']]

augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query

In [29]:
print(augmented_query)

for full documentation on:\n\nGetting started (installation, setting up the environment, simple examples)\n\nHow-To examples (demos, integrations, helper functions)\n\nReference (full API docs)\n\nResources (high-level explanation of core concepts)\n\nð\x9f\x9a\x80 What can this help with?\n\nThere are six main areas that LangChain is designed to help with.\nThese are, in increasing order of complexity:\n\nð\x9f“\x83 LLMs and Prompts:\n\nThis includes prompt management, prompt optimization, a generic interface for all LLMs, and common utilities for working with LLMs.\n\nð\x9f”\x97 Chains:\n\nChains go beyond a single LLM call and involve sequences of calls (whether to an LLM or a different utility). LangChain provides a standard interface for chains, lots of integrations with other tools, and end-to-end chains for common applications.\n\nð\x9f“\x9a Data Augmented Generation:\n\nData Augmented Generation involves specific types of chains that first interact with an external data source 

Now we ask the question:

In [30]:
# system message to 'prime' the model
primer = f"""You are Q&A bot. A highly intelligent system that answers
user questions based on the information provided by the user above
each question. If the information can not be found in the information
provided by the user you truthfully say "I don't know".
"""

res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ]
)

To display this response nicely, we will display it in markdown.

In [31]:
from IPython.display import Markdown

display(Markdown(res['choices'][0]['message']['content']))

To use the LLMChain in LangChain, you must follow these general steps:

1. Install and set up the environment: Ensure that you've installed LangChain and set up the required environment.
2. Create an LLMChain instance: Start by creating an instance of the LLMChain, specifying the desired LLMs and other configuration as needed.
3. Define your input and desired use case: Identify the input text or prompts you want to use, and the specific use case for the LLMChain (e.g., question-answering, data-augmented generation, etc.).
4. Run the LLMChain: Execute the LLMChain with the specified input and use case. This will result in a sequence of calls to the LLM or other utilities as defined in the chain.
5. Evaluate or interact with the output: Analyze the output generated by the LLMChain and use it in your application as needed.

Refer to the LangChain documentation for more specific instructions and examples related to LLMChains and how to use them for different applications and integrations.

Let's compare this to a non-augmented query...

In [32]:
res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": query}
    ]
)
display(Markdown(res['choices'][0]['message']['content']))

I don't know.

If we drop the `"I don't know"` part of the `primer`?

In [33]:
res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are Q&A bot. A highly intelligent system that answers user questions"},
        {"role": "user", "content": query}
    ]
)
display(Markdown(res['choices'][0]['message']['content']))

LLMChain (Language Model Layered Chain) in LangChain is a machine learning technique that provides an improved language model for natural language processing tasks. To use LLMChain in LangChain, follow these steps:

1. Install LangChain: First, ensure that you have installed the necessary LangChain library on your local machine or server. You can refer to the official documentation or GitHub repository for installation instructions and any required dependencies.

2. Import the necessary modules: In your Python script, import the required modules from LangChain, including the LLMChain class:

```python
from langchain.llmchain import LLMChain
```

3. Instantiate the LLMChain class: Create an instance of the LLMChain class, providing any necessary parameters or options:

```python
llmchain = LLMChain()
```

4. Preprocess and tokenize data: In order to work with text data in LangChain, you'll need to preprocess and tokenize it. You can use built-in functions provided by the library or other NLP libraries like NLTK or SpaCy:

```python
preprocessed_data = preprocess_your_data(input_text)
tokens = tokenize(preprocessed_data)
```

5. Train the LLMChain: Train the LLMChain instance with your prepared tokens. This will enable the model to learn patterns and relationships between words:

```python
llmchain.train(tokens)
```

6. Generate language samples: After training, you can use the LLMChain instance to generate language samples based on the learned patterns:

```python
generated_samples = llmchain.generate(num_samples, seed_words, max_length)
```

7. Evaluate and fine-tune the model: You can examine the generated samples and tweak the LLMChain parameters or options to achieve better results. You may also consider training the model on different or additional datasets to improve its performance.

Remember that the actual implementation may vary depending on your specific use case and requirements, so refer to the LangChain documentation and examples to learn more about using LLMChain effectively.

Then we see something even worse than `"I don't know"` — hallucinations. Clearly augmenting our queries with additional context can make a huge difference to the performance of our system.

Great, we've seen how to augment GPT-4 with semantic search to allow us to answer LangChain specific queries.

Once you're finished, we delete the index to save resources.

In [34]:
pinecone.delete_index(index_name)

---